In [1]:
'''
Work
請改變 reduce_lr 的 patience 和 factor 並比較不同設定下，對訓練/驗證集的影響
請將 optimizer 換成 Adam、RMSprop 搭配 reduce_lr 並比較訓練結果
'''
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
'''
def build_mlp(input_shape, output_units=10, num_neurons=[256, 256, 256]):
    """Code Here
    建立你的神經網路
    """
    return model
'''
from keras.layers import BatchNormalization

"""
建立神經網路，並加入 BN layer
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [7]:
LEARNING_RATE = 1e-3
EPOCHS = 25 # IF you feel too run to finish, try to make it smaller
BATCH_SIZE = 1024 

In [9]:
# 載入 Callbacks
from keras.callbacks import ReduceLROnPlateau

optimizer_set = [keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=0.95),
                 keras.optimizers.Adam(lr=LEARNING_RATE),
                 keras.optimizers.RMSprop(lr=LEARNING_RATE)]

"""Code Here
建立實驗的比較組合
"""
reduce_lr_factor = [0.5, 0.1]
redice_lr_patient = [5, 2]

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
import itertools #enumerate(sequence, start=0)，返回一个枚举对象。sequence必须是序列或迭代器iterator，或者支持迭代的对象。enumerate()返回对象的每个元素都是一个元组，每个元组包括两个值，一个是计数，一个是sequence的值，计数是从start开始的，start默认为0。
'''
enumerate()是python的内置函数
enumerate在字典上是枚举、列举的意思
对于一个可迭代的（iterable）/可遍历的对象（如列表、字符串），enumerate将其组成一个索引序列，利用它可以同时获得索引和值
enumerate多用于在for循环中得到计数
例如对于一个seq，得到：
enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标，一般用在 for 循环当中。
　itertools.product(sequence，repeat) 　 #從sequence中拿出repeat個數做排列（repeat關鍵字傳參） 有放回的拿出  repeat與sequence的長度無關。
 itertools.product
 上面的是获取相加的结果，下面的是获取相乘的结果
 a = itertools.product('浪子', '好帅')
print(list(a))
返回结果：

[('浪', '好'), ('浪', '帅'), ('子', '好'), ('子', '帅')]
'''
results = {}
         '''
         optim, reduce_factor, reduce_patient 這三個變數在 itertools.product(optimizer_set, reduce_lr_factor, redice_lr_patie
         '''
    
for i, (optim, reduce_factor, reduce_patient) in enumerate(itertools.product(optimizer_set, reduce_lr_factor, redice_lr_patient)):
    print("Numbers of exp: %i, reduce_factor: %.2f, reduce_patient: %i" % (i, reduce_factor, reduce_patient))
    model = build_mlp(input_shape=x_train.shape[1:]) ##多层感知机（MLP） #reshape（行，列）可以根据指定的数值将数据转换为特定的行数和列数，这个好理解，就是转换成矩阵。
    model.summary() #summary() #模型摘要
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optim)
    #compile() 函数将一个字符串编译为字节代码。
    #亦称作多类的对数损失，注意使用该目标函数时，需要将标签转化为形如(nb_samples, nb_classes)的二值序列
    #metrics 指標= accuracy"準確性
    #optimizer 優化器
    
    """Code Here
    設定 reduce learning rate 的 callback function
    """
    #ReduceLROnPlateau 当指标变化小时，减少学习率
    reduce_lr = ReduceLROnPlateau(factor=reduce_factor, #因子
                              min_lr=1e-12, 
                              monitor='val_loss', 
                              patience=reduce_patient,  #patience: 当early stop被激活(如发现loss相比上一个epoch训练没有下降)，则经过patience个epoch后停止训练
                              verbose=1)
    
    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True,##shuffle：是否把數據隨機打亂之後再進行訓練
              callbacks=[reduce_lr]
             )

    # Collect results
    exp_name_tag = ("exp-%s" % (i))
    results[exp_name_tag] = {'train-loss': model.history.history["loss"],
                             'valid-loss': model.history.history["val_loss"],
                             'train-acc': model.history.history["acc"],
                             'valid-acc': model.history.history["val_acc"]}

Numbers of exp: 0, reduce_factor: 0.50, reduce_patient: 5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalizatio

Epoch 11/25
50000/50000 [==============================] - 8s 170us/step - loss: 1.3977 - acc: 0.5139 - val_loss: 1.5489 - val_acc: 0.4554
Epoch 12/25
50000/50000 [==============================] - 9s 184us/step - loss: 1.3746 - acc: 0.5228 - val_loss: 1.5361 - val_acc: 0.4594
Epoch 13/25
50000/50000 [==============================] - 9s 172us/step - loss: 1.3541 - acc: 0.5309 - val_loss: 1.5285 - val_acc: 0.4586
Epoch 14/25
50000/50000 [==============================] - 9s 171us/step - loss: 1.3351 - acc: 0.5388 - val_loss: 1.5252 - val_acc: 0.4600
Epoch 15/25
50000/50000 [==============================] - 8s 168us/step - loss: 1.3163 - acc: 0.5442 - val_loss: 1.5127 - val_acc: 0.4660
Epoch 16/25
50000/50000 [==============================] - 8s 168us/step - loss: 1.2983 - acc: 0.5507 - val_loss: 1.5116 - val_acc: 0.4705
Epoch 17/25
50000/50000 [==============================] - 9s 185us/step - loss: 1.2807 - acc: 0.5579 - val_loss: 1.5057 - val_acc: 0.4703
Epoch 18/25
50000/50000 [==

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 12s 234us/step - loss: 2.3588 - acc: 0.2355 - val_loss: 2.2318 - val_acc: 0.2826
Epoch 2/25
50000/50000 [==============================] - 8s 169us/step - loss: 1.8650 - acc: 0.3572 - val_loss: 1.9293 - val_acc: 0.3511
Epoch 3/25
50000/50000 [==============================] - 8s 161us/step - loss: 1.7229 - acc: 0.4017 - val_loss: 1.7885 - val_acc: 0.3893
Epoch 4/25
50000/50000 [==============================] - 9s 188us/step - loss: 1.6470 - acc: 0.4258 - val_loss: 1.7174 - val_acc: 0.4027
Epoch 5/25
50000/50000 [==============================] - 9s 172us/step - loss: 1.5937 - acc: 0.4431 - val_loss: 1.6739 - val_acc: 0.4151
Epoch 6/25
50000/50000 [==============================] - 9s 181us/step - loss: 1.5513 - acc: 0.4590 - val_loss: 1.6393 - val_acc: 0.4274
Epoch 7/25
50000/50000 [==============================] - 8s 166us/step - loss: 1.5148 - acc: 0.4710 - val_loss: 1.6270 -

50000/50000 [==============================] - 9s 175us/step - loss: 0.4971 - acc: 0.8466 - val_loss: 1.5390 - val_acc: 0.5045
Epoch 22/25
50000/50000 [==============================] - 8s 163us/step - loss: 0.4769 - acc: 0.8552 - val_loss: 1.5800 - val_acc: 0.5090
Epoch 23/25
50000/50000 [==============================] - 9s 173us/step - loss: 0.4541 - acc: 0.8640 - val_loss: 1.5965 - val_acc: 0.5051

Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 24/25
50000/50000 [==============================] - 9s 171us/step - loss: 0.4049 - acc: 0.8870 - val_loss: 1.5458 - val_acc: 0.5234
Epoch 25/25
50000/50000 [==============================] - 9s 171us/step - loss: 0.3856 - acc: 0.8960 - val_loss: 1.5738 - val_acc: 0.5231
Numbers of exp: 5, reduce_factor: 0.50, reduce_patient: 2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 3072)     

Epoch 5/25
50000/50000 [==============================] - 9s 183us/step - loss: 1.7950 - acc: 0.3817 - val_loss: 1.8525 - val_acc: 0.3658
Epoch 6/25
50000/50000 [==============================] - 9s 172us/step - loss: 1.7609 - acc: 0.3926 - val_loss: 1.8251 - val_acc: 0.3729
Epoch 7/25
50000/50000 [==============================] - 9s 170us/step - loss: 1.7328 - acc: 0.4007 - val_loss: 1.8034 - val_acc: 0.3788
Epoch 8/25
50000/50000 [==============================] - 8s 168us/step - loss: 1.7088 - acc: 0.4075 - val_loss: 1.7839 - val_acc: 0.3855
Epoch 9/25
50000/50000 [==============================] - 8s 159us/step - loss: 1.6880 - acc: 0.4149 - val_loss: 1.7685 - val_acc: 0.3908
Epoch 10/25
50000/50000 [==============================] - 8s 168us/step - loss: 1.6692 - acc: 0.4216 - val_loss: 1.7542 - val_acc: 0.3936
Epoch 11/25
50000/50000 [==============================] - 9s 175us/step - loss: 1.6541 - acc: 0.4266 - val_loss: 1.7428 - val_acc: 0.3996
Epoch 12/25
50000/50000 [=======

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 11s 224us/step - loss: 1.8061 - acc: 0.3727 - val_loss: 2.1172 - val_acc: 0.3135
Epoch 2/25
50000/50000 [==============================] - 8s 161us/step - loss: 1.5450 - acc: 0.4515 - val_loss: 2.2465 - val_acc: 0.2886
Epoch 3/25
50000/50000 [==============================] - 8s 162us/step - loss: 1.4406 - acc: 0.4909 - val_loss: 2.8880 - val_acc: 0.2202
Epoch 4/25
50000/50000 [==============================] - 8s 163us/step - loss: 1.3657 - acc: 0.5152 - val_loss: 3.1774 - val_acc: 0.2291
Epoch 5/25
50000/50000 [==============================] - 8s 163us/step - loss: 1.2996 - acc: 0.5409 - val_loss: 1.8053 - val_acc: 0.3705
Epoch 6/25
50000/50000 [==============================] - 8s 165us/step - loss: 1.2466 - acc: 0.5588 - val_loss: 1.9858 - val_acc: 0.3364
Epoch 7/25
50000/50000 [==============================] - 8s 161us/step - loss: 1.2080 - acc: 0.5739 - val_loss: 3.9785 -

50000/50000 [==============================] - 8s 168us/step - loss: 0.9274 - acc: 0.6944 - val_loss: 1.4035 - val_acc: 0.5175
Epoch 21/25
50000/50000 [==============================] - 8s 158us/step - loss: 0.9204 - acc: 0.6993 - val_loss: 1.4079 - val_acc: 0.5076

Epoch 00021: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 22/25
50000/50000 [==============================] - 8s 162us/step - loss: 0.9078 - acc: 0.7054 - val_loss: 1.4023 - val_acc: 0.5149
Epoch 23/25
50000/50000 [==============================] - 8s 164us/step - loss: 0.9042 - acc: 0.7072 - val_loss: 1.4013 - val_acc: 0.5193
Epoch 24/25
50000/50000 [==============================] - 8s 164us/step - loss: 0.9007 - acc: 0.7078 - val_loss: 1.4040 - val_acc: 0.5142
Epoch 25/25
50000/50000 [==============================] - 8s 162us/step - loss: 0.8976 - acc: 0.7096 - val_loss: 1.4020 - val_acc: 0.5170

Epoch 00025: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Numbers of exp: 

50000/50000 [==============================] - 8s 161us/step - loss: 2.0243 - acc: 0.3139 - val_loss: 2.0464 - val_acc: 0.3065
Epoch 7/25
50000/50000 [==============================] - 8s 164us/step - loss: 1.9764 - acc: 0.3278 - val_loss: 1.9985 - val_acc: 0.3209
Epoch 8/25
50000/50000 [==============================] - 8s 166us/step - loss: 1.9359 - acc: 0.3381 - val_loss: 1.9660 - val_acc: 0.3273
Epoch 9/25
50000/50000 [==============================] - 8s 164us/step - loss: 1.9018 - acc: 0.3487 - val_loss: 1.9358 - val_acc: 0.3364
Epoch 10/25
50000/50000 [==============================] - 8s 164us/step - loss: 1.8730 - acc: 0.3561 - val_loss: 1.9103 - val_acc: 0.3440
Epoch 11/25
50000/50000 [==============================] - 8s 163us/step - loss: 1.8465 - acc: 0.3641 - val_loss: 1.8878 - val_acc: 0.3514
Epoch 12/25
50000/50000 [==============================] - 8s 164us/step - loss: 1.8230 - acc: 0.3721 - val_loss: 1.8694 - val_acc: 0.3556
Epoch 13/25
50000/50000 [=================